### 中醫醫案症狀詞抽取
- model: CRF
- 主症與兼症合併
- 詞位標注: OSBIE
- 671個詞沒有完全對應(皆標為O)
- postag使用nltk
- 以句點為分隔拆句子

In [6]:
import json
import nltk

In [7]:
def getTagListWithString(str):
    list1 = []
    str = str.replace('，',' ')
    str = str.replace('、',' ')
    str = str.replace(',',' ')
    str_list = str.split(" ")
    
    for s in str_list:
        s = s.strip()
        if len(s) > 0:
            list1.append(s)
    
    #list2 = list(filter(None, second_list))

    return list1
    

In [8]:
def getLabelWithWordIndex(i,ne_index_list):
    for (ne_idx,ne_len) in ne_index_list: 
        if i == ne_idx:
            return "B-S"
        elif i > ne_idx and i < (ne_idx + ne_len - 1):
            return "I-S"
        elif i == (ne_idx + ne_len - 1):
            return "E-S"
            
    return "O"

In [9]:
filename = "case_summary.csv"
with open(filename) as f:
    lines = f.readlines()

docs = []
c = 0
t = 0
for idx,line in enumerate(lines):
    cols = line.strip().split(",")
    case_content = cols[2].replace(" ","")
    sentences = case_content.split("。")
    
    summary_str = cols[3].strip()
    
    if idx == 0 or len(summary_str) == 0 :
        continue
        
    #處理每個doc的summary(主症和兼症)
    summary_str = summary_str.strip('"')
    summary_str = summary_str.replace('""','"')
    summary_str = summary_str.replace('#',',')
    #print(summary_str)
        
    summary_json = json.loads(summary_str)
        
    keys = summary_json.keys()
    if "main" in keys:
        main = summary_json["main"]
        main_list = getTagListWithString(main)
        #print(main_list)
    if "second" in keys:
        second = summary_json["second"]
        second_list = getTagListWithString(second)
        #print(second_list)
        
    if len(second_list) > 0:
        main_list.extend(second_list)
    
    t += len(main_list)
        
        
    for sent in sentences:
        ne_index_list = []
        for ne in main_list:
            ne_idx = sent.find(ne)
            if ne_idx != -1 :
                ne_index_list.append((ne_idx,len(ne)))

        texts = []
        for i,w in enumerate(sent):
            label = getLabelWithWordIndex(i,ne_index_list)
            texts.append((w,label))

        docs.append(texts)

print(len(docs))

48901


In [12]:
import pynlpir  # 引入依赖包
pynlpir.open()  # 打开分词器

In [29]:
data = []

for i, doc in enumerate(docs):
    tokens = [t for t, label in doc]
    #print(tokens)
    tagged = []
    for tk in tokens:
        tg = pynlpir.segment(tk)
        tagged.append(tg[0])
    #tagged = nltk.pos_tag(tokens)
    if 'None' in tagged:
        print(tokens)
        print(tagged)
    data.append([(w, pos, label) for (w, label), (word, pos) in zip(doc, tagged)])

part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gms'
part of speech not recognized: 'gjtgj'
part of speech not recognized: 'gms'
part of speech not recognized: 'gm

In [19]:
print(len(data))

48901


In [58]:
def word2features(doc, i):
    word = doc[i][0]
    postag = doc[i][1]
    if postag is None:
            postag = 'none'

    # Common features for all words
    features = [
        'bias',
        'word.lower=' + word.lower(),
        'word[-3:]=' + word[-3:],
        'word[-2:]=' + word[-2:],
        'word.isupper=%s' % word.isupper(),
        'word.istitle=%s' % word.istitle(),
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag
    ]

    # Features for words that are not
    # at the beginning of a document
    if i > 0:
        word1 = doc[i-1][0]
        postag1 = doc[i-1][1]
        if postag1 is None:
            postag1 = 'none'
            
        features.extend([
            '-1:word.lower=' + word1.lower(),
            '-1:word.istitle=%s' % word1.istitle(),
            '-1:word.isupper=%s' % word1.isupper(),
            '-1:word.isdigit=%s' % word1.isdigit(),
            '-1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'beginning of a document'
        features.append('BOS')

    # Features for words that are not
    # at the end of a document
    if i < len(doc)-1:
        word1 = doc[i+1][0]
        postag1 = doc[i+1][1]
        if postag1 is None:
            postag1 = 'none'
       
        features.extend([
            '+1:word.lower=' + word1.lower(),
            '+1:word.istitle=%s' % word1.istitle(),
            '+1:word.isupper=%s' % word1.isupper(),
            '+1:word.isdigit=%s' % word1.isdigit(),
            '+1:postag=' + postag1
        ])
    else:
        # Indicate that it is the 'end of a document'
        features.append('EOS')

    return features

In [59]:
from sklearn.model_selection import train_test_split

# A function for extracting features in documents
def extract_features(doc):
    return [word2features(doc, i) for i in range(len(doc))]

# A function fo generating the list of labels for each document
def get_labels(doc):
    return [label for (token, postag, label) in doc]

X = [extract_features(doc) for doc in data]
y = [get_labels(doc) for doc in data]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [60]:
print(len(X_train))
print(len(X_test))

39120
9781


In [61]:
import pycrfsuite
trainer = pycrfsuite.Trainer(verbose=True)

# Submit training data to the trainer
for xseq, yseq in zip(X_train, y_train):
    trainer.append(xseq, yseq)

# Set the parameters of the model
trainer.set_params({
    # coefficient for L1 penalty
    'c1': 0.1,

    # coefficient for L2 penalty
    'c2': 0.01,  

    # maximum number of iterations
    'max_iterations': 200,

    # whether to include transitions that
    # are possible, but not observed
    'feature.possible_transitions': True
})

# Provide a file name as a parameter to the train function, such that
# the model will be saved to the file when training is finished
trainer.train('crf.model')

Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 1
0....1....2....3....4....5....6....7....8....9....10
Number of features: 30273
Seconds required: 3.016

L-BFGS optimization
c1: 0.100000
c2: 0.010000
num_memories: 6
max_iterations: 200
epsilon: 0.000010
stop: 10
delta: 0.000010
linesearch: MoreThuente
linesearch.max_iterations: 20

***** Iteration #1 *****
Loss: 288445.059214
Feature norm: 1.000000
Error norm: 93975.942046
Active features: 30123
Line search trials: 1
Line search step: 0.000000
Seconds required for this iteration: 1.116

***** Iteration #2 *****
Loss: 285985.412443
Feature norm: 1.018367
Error norm: 79290.899746
Active features: 28552
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.566

***** Iteration #3 *****
Loss: 276698.330538
Feature norm: 1.113522
Error norm: 72196.407513
Active features: 25448
Line search trials: 1
Line search step: 1.000000
Seconds required 

***** Iteration #39 *****
Loss: 43611.814273
Feature norm: 33.114756
Error norm: 16015.283267
Active features: 24115
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.567

***** Iteration #40 *****
Loss: 43368.432454
Feature norm: 34.702496
Error norm: 44786.015801
Active features: 23907
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.565

***** Iteration #41 *****
Loss: 42637.597806
Feature norm: 34.817695
Error norm: 5391.090425
Active features: 24351
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #42 *****
Loss: 42302.227226
Feature norm: 35.346628
Error norm: 4334.624422
Active features: 24344
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.568

***** Iteration #43 *****
Loss: 41550.297933
Feature norm: 37.534293
Error norm: 11627.104817
Active features: 23815
Line search trials: 1
Line search step: 1.000

***** Iteration #81 *****
Loss: 34206.148757
Feature norm: 92.516491
Error norm: 5542.994885
Active features: 21952
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.564

***** Iteration #82 *****
Loss: 34160.156688
Feature norm: 93.557243
Error norm: 7826.913792
Active features: 21856
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #83 *****
Loss: 34115.358514
Feature norm: 94.768702
Error norm: 6537.195407
Active features: 21859
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.560

***** Iteration #84 *****
Loss: 34078.556179
Feature norm: 95.297403
Error norm: 4880.788740
Active features: 21893
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.560

***** Iteration #85 *****
Loss: 34042.805433
Feature norm: 96.025052
Error norm: 4023.861775
Active features: 21827
Line search trials: 1
Line search step: 1.000000

***** Iteration #121 *****
Loss: 33430.202530
Feature norm: 109.517750
Error norm: 4006.016722
Active features: 21165
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.560

***** Iteration #122 *****
Loss: 33418.189918
Feature norm: 109.655575
Error norm: 2107.828744
Active features: 21148
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #123 *****
Loss: 33414.504778
Feature norm: 109.869736
Error norm: 3603.199068
Active features: 21137
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.563

***** Iteration #124 *****
Loss: 33403.549381
Feature norm: 109.996448
Error norm: 1936.007371
Active features: 21112
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.560

***** Iteration #125 *****
Loss: 33400.156203
Feature norm: 110.216781
Error norm: 3559.055886
Active features: 21112
Line search trials: 1
Line search step

***** Iteration #164 *****
Loss: 33199.967760
Feature norm: 115.674324
Error norm: 1693.523803
Active features: 20835
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.566

***** Iteration #165 *****
Loss: 33197.227664
Feature norm: 115.829717
Error norm: 2687.206900
Active features: 20812
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.583

***** Iteration #166 *****
Loss: 33189.830393
Feature norm: 115.921306
Error norm: 1415.240178
Active features: 20811
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.561

***** Iteration #167 *****
Loss: 33188.762179
Feature norm: 116.090519
Error norm: 2818.522575
Active features: 20824
Line search trials: 1
Line search step: 1.000000
Seconds required for this iteration: 0.560

***** Iteration #168 *****
Loss: 33181.804609
Feature norm: 116.193717
Error norm: 1995.983311
Active features: 20809
Line search trials: 1
Line search step

In [62]:
tagger = pycrfsuite.Tagger()
tagger.open('crf.model')
y_pred = [tagger.tag(xseq) for xseq in X_test]

# Let's take a look at a random sample in the testing set
i = 12
for x, y in zip(y_pred[i], [x[1].split("=")[1] for x in X_test[i]]):
    print("%s (%s)" % (y, x))

脉 (O)
细 (O)
， (O)
舌 (O)
胖 (O)
大 (O)
， (O)
舌 (O)
苔 (O)
黑 (O)
润 (O)


In [63]:
import numpy as np
from sklearn.metrics import classification_report

# Create a mapping of labels to indices
labels = {"O": 0,"B-S": 1,"I-S": 2,"E-S": 3}

# Convert the sequences of tags into a 1-dimensional array
predictions = np.array([labels[tag] for row in y_pred for tag in row])
truths = np.array([labels[tag] for row in y_test for tag in row])

# Print out the classification report
print(classification_report(
    truths, predictions,
    target_names=["O", "B-S","I-S","E-S"]))

             precision    recall  f1-score   support

          O       0.97      0.99      0.98    304690
        B-S       0.60      0.46      0.52      4284
        I-S       0.50      0.31      0.38      5028
        E-S       0.61      0.46      0.52      4282

avg / total       0.96      0.96      0.96    318284



In [5]:
s = '舌苔白'


[('舌苔', 'noun'), ('白', 'adjective')]